<a href="https://colab.research.google.com/github/veronica-araoz/veronica-araoz_clustering/blob/main/Clustering_global.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# 1) Carga de datos (Google Colab + Google Drive)
# ============================================================
# Este notebook fue ejecutado en Google Colab.
# Si se trabaja en Colab, se puede montar Google Drive para acceder al dataset.

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import pandas as pd

# Ruta al dataset (archivo privado en Google Drive; no se incluye en el repositorio)
DATASET_PATH = "/content/drive/MyDrive/CLUSTERING/cluster_global.xlsx"

df = pd.read_excel(DATASET_PATH)

print(f"✅ Archivo cargado correctamente: {df.shape[0]} filas, {df.shape[1]} columnas")
print("Columnas disponibles:", df.columns.tolist())


### 1. Setup y preparación del corpus
Instalación de dependencias (Colab) y creación de una columna unificada de texto para el análisis

In [ ]:
# ============================================================
# 1) Setup + preparación del corpus
# ============================================================

# Instalación de dependencias (solo necesario en Google Colab)
!pip -q install transformers sentence_transformers umap-learn
!pip -q install --upgrade scikit-learn openpyxl

import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt

from sentence_transformers import SentenceTransformer
from umap import UMAP
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# ---------------------
# Recursos NLTK
# ---------------------
try:
    nltk.data.find("corpora/stopwords")
except LookupError:
    nltk.download("stopwords")

try:
    nltk.data.find("tokenizers/punkt")
except LookupError:
    nltk.download("punkt")

try:
    nltk.data.find("corpora/wordnet")
except LookupError:
    nltk.download("wordnet")

# ---------------------
# Carga de datos
# ---------------------
DATASET_PATH = "/content/drive/MyDrive/CLUSTERING/cluster_global.xlsx"  # archivo privado (no incluido en el repo)
df = pd.read_excel(DATASET_PATH)

# ---------------------
# Preparación de texto
# ---------------------
df["Titular"] = df["Titular"].fillna("").astype(str)
df["Bajada"] = df["Bajada"].fillna("").astype(str)
df["Cuerpo_texto"] = df["Cuerpo_texto"].fillna("").astype(str)

df["texto_completo"] = df[["Titular", "Bajada", "Cuerpo_texto"]].agg(" ".join, axis=1)

print("✅ Paso 1 completado: datos cargados y columna 'texto_completo' generada.")
print("Columnas disponibles:", df.columns.tolist())
print("\nEjemplo de texto combinado:\n", df["texto_completo"].iloc[0][:500])


### 2.  Embeddings + reducción de dimensionalidad (UMAP)
Se generan embeddings semánticos a nivel documento con SentenceTransformer (modelo multilingüe) a partir de la columna texto_completo. Luego se aplica UMAP para reducir dimensionalidad y facilitar el clustering.

In [ ]:
# ============================================================
# 2) Embeddings + reducción de dimensionalidad (UMAP)
# ============================================================

# ---------------------
# Embeddings (SentenceTransformer)
# ---------------------
MODEL_NAME = "distiluse-base-multilingual-cased-v1"  # modelo multilingüe (adecuado para español)
model = SentenceTransformer(MODEL_NAME)

texts = df["texto_completo"].fillna("").tolist()
embeddings = model.encode(texts, show_progress_bar=True)

print("✅ Embeddings generados.")
print("Forma de la matriz de embeddings:", embeddings.shape)  # (n_notas, 512)

# ---------------------
# UMAP para reducción de dimensionalidad
# ---------------------
# Se reduce a 5 dimensiones para facilitar el clustering manteniendo estructura semántica.
reducer = UMAP(
    n_neighbors=15,
    min_dist=0.1,
    n_components=5,
    metric="cosine",
    random_state=42
)

reduced_embeddings = reducer.fit_transform(embeddings)

print("✅ Dimensionalidad reducida con UMAP.")
print("Forma de la matriz reducida:", reduced_embeddings.shape)  # (n_notas, 5)


### 3.  Selección del número de clústeres (K)
Se evalúan distintos valores de K mediante el coeficiente de silueta y el método del codo (inercia) para fundamentar la elección final.

In [ ]:
# ============================================================
# 3) Selección de K (Silueta + Codo)
# ============================================================

K_MIN = 2
K_MAX = 10
k_range = range(K_MIN, K_MAX)

silhouettes = []
inercia = []

print("\n--- Evaluación de K ---")
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(reduced_embeddings)

    sil_score = silhouette_score(reduced_embeddings, labels)
    silhouettes.append(sil_score)

    inercia.append(kmeans.inertia_)

    print(f"K={k} | Silueta={sil_score:.4f} | Inercia={kmeans.inertia_:.2f}")

best_k_silhouette = list(k_range)[int(np.argmax(silhouettes))]
print(f"\n✅ Mejor K según Silueta: {best_k_silhouette}")

# ---------------------
# Gráfico: Método del Codo
# ---------------------
plt.figure(figsize=(10, 6))
plt.plot(list(k_range), inercia, marker="o")
plt.title("Método del Codo (corpus completo)")
plt.xlabel("Número de clústeres (K)")
plt.ylabel("Inercia")
plt.xticks(list(k_range))
plt.grid(True)
plt.show()

# ---------------------
# Gráfico: Silueta
# ---------------------
plt.figure(figsize=(10, 6))
plt.plot(list(k_range), silhouettes, marker="o")
plt.title("Método de la Silueta (corpus completo)")
plt.xlabel("Número de clústeres (K)")
plt.ylabel("Silueta")
plt.xticks(list(k_range))
plt.grid(True)
plt.show()


Interpretación de la cantidad de clústeres (selección de k)

Para decidir el número óptimo de clústeres (k), se utilizaron dos criterios complementarios: coeficiente de silueta y método del codo (inercia).

1) Coeficiente de silueta

El coeficiente de silueta evalúa simultáneamente cohesión interna (qué tan similares son los elementos dentro de un clúster) y separación entre clústeres.

Valores cercanos a 1 → clústeres bien definidos y separados

Valores cercanos a 0 → clústeres superpuestos o poco distinguibles

Valores negativos → asignación deficiente (elementos más cercanos a otros clústeres que al propio)

➡️ En este caso, la silueta máxima fue 0.7556 con k = 2, lo que sugiere que una partición en dos clústeres produce la estructura más consistente y separada del corpus.

2) Método del codo (inercia)

La inercia mide la suma de distancias de los puntos al centroide de su clúster. A medida que aumenta k, la inercia disminuye, pero el objetivo es identificar un punto a partir del cual la mejora se vuelve marginal (el “codo”).

En este análisis:

La inercia disminuye de forma continua al aumentar k.

La caída más marcada se observa al pasar de k = 2 a k = 3 (aprox. 907 puntos).

A partir de k = 3–4, la reducción se vuelve más gradual.

➡️ Esto indica que k = 2 o k = 3 pueden representar un buen equilibrio entre simplicidad interpretativa y capacidad explicativa del modelo.

### 4. Clustering final con K-Means
Se entrena un modelo K-Means con el valor de k seleccionado y se asigna una etiqueta de clúster a cada nota del corpus.

In [ ]:
# ============================================================
# 4) Clustering final (K-Means)
# ============================================================

k_final = 2  # valor seleccionado según silueta y método del codo

kmeans_final = KMeans(n_clusters=k_final, random_state=42, n_init=10)
df["cluster_label"] = kmeans_final.fit_predict(reduced_embeddings)

print(f"\n✅ Conteo de notas por clúster (K={k_final}):")
print(df["cluster_label"].value_counts().sort_index())

# ---------------------
# Guardar resultados
# ---------------------
import os

OUTPUT_DIR = "/content/outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

output_path = os.path.join(OUTPUT_DIR, "clusters_global.xlsx")
df.to_excel(output_path, index=False)

print(f"✅ Resultados guardados en: {output_path}")


### Visualizaciones
Gráfico de dispersión en 2D (UMAP)

Para facilitar la interpretación de los resultados, se realiza una reducción adicional a 2 dimensiones mediante UMAP y se visualizan las notas periodísticas en un plano 2D, coloreadas según la etiqueta de clúster asignada por K-Means. Esta visualización permite explorar la separación (o superposición) entre agrupamientos y detectar patrones generales en el corpus.

In [ ]:
# =====================
# Visualización en 2D con UMAP (flexible para cualquier número de clusters)
# =====================
import matplotlib.pyplot as plt
import seaborn as sns
from umap import UMAP

sns.set_style("whitegrid")

clusters_unicos = sorted(df["cluster_label"].unique())
n_clusters = len(clusters_unicos)

reducer_2d = UMAP(n_components=2, random_state=42, metric="cosine")
embeddings_2d = reducer_2d.fit_transform(reduced_embeddings)

plt.figure(figsize=(10, 8))
sns.scatterplot(
    x=embeddings_2d[:, 0],
    y=embeddings_2d[:, 1],
    hue=df["cluster_label"],
    palette="tab10",
    s=50,
    alpha=0.7
)

plt.title(f"Visualización de Clústeres en 2D - Corpus Global (k={n_clusters})")
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")
plt.legend(title="Cluster", loc="best")
plt.tight_layout()

output_path_fig = "/content/clusters_global_2D.png"
plt.savefig(output_path_fig, dpi=300, bbox_inches="tight")
print(f"✅ Gráfico guardado en: {output_path_fig}")

plt.show()


## Interpretación de clusters: Nubes de palabras (WordCloud)

Para explorar el contenido temático de cada clúster, se generan **nubes de palabras** a partir del texto completo (`Titular + Bajada + Cuerpo_texto`) de las notas asignadas a cada agrupamiento.

📌 **Nota metodológica:** se eliminan stopwords en español y un conjunto de términos irrelevantes frecuentes en textos periodísticos, con el fin de resaltar palabras con mayor capacidad descriptiva.

In [ ]:
# Si estás en Google Colab y no tenés instalada la librería:
# !pip install wordcloud

import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import defaultdict
import nltk
from nltk.corpus import stopwords
import re

# =====================
# Recursos NLTK
# =====================
try:
    nltk.data.find("corpora/stopwords")
except:
    nltk.download("stopwords")

# =====================
# Stopwords + términos irrelevantes (ajustables)
# =====================
palabras_irrelevantes = [
    "noticias", "relacionadas", "ver", "más", "además", "así",
    "comentarios", "nan", "aunque", "solo", "uno", "aun"
]

stop_words = set(stopwords.words("spanish"))
stop_words.update(palabras_irrelevantes)

print("\n--- Generando nubes de palabras por clúster (corpus global) ---")

# =====================
# Agrupar textos por clúster
# =====================
cluster_texts = df.groupby("cluster_label")["texto_completo"].apply(list).to_dict()

for cluster_id, texts in cluster_texts.items():
    all_text = " ".join(texts).lower()

    # Preprocesamiento básico
    all_text = re.sub(r"\d+", "", all_text)  # eliminar números
    all_text = re.sub(r"[^\w\s]", " ", all_text)  # eliminar signos
    all_text = re.sub(r"\s+", " ", all_text).strip()  # limpiar espacios

    # Filtrar stopwords y palabras cortas
    all_text = " ".join([
        word for word in all_text.split()
        if word not in stop_words and len(word) > 2
    ])

    # WordCloud
    wordcloud = WordCloud(
        width=900,
        height=450,
        background_color="white",
        collocations=False
    ).generate(all_text)

    # Plot
    plt.figure(figsize=(12, 6))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"Nube de palabras - Clúster {cluster_id} (Corpus Global)")
    plt.tight_layout()
    plt.show()


## Interpretación de clusters: palabras clave y palabras distintivas

Además de la visualización, se realiza una exploración lexical para caracterizar cada clúster:

- **Palabras clave:** términos más frecuentes dentro de cada clúster (Top 10).
- **Palabras distintivas:** términos que aparecen exclusivamente en un clúster y no en los demás, ordenados por frecuencia.

Este paso permite una aproximación rápida a los ejes temáticos predominantes en cada agrupamiento.


In [ ]:
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
from IPython.display import display

# =====================
# Preparar stopwords
# =====================
try:
    nltk.data.find("corpora/stopwords")
except:
    nltk.download("stopwords")

palabras_irrelevantes = [
    "noticias", "relacionadas", "ver", "más", "además", "así",
    "comentarios", "nan", "aunque", "solo", "sólo", "ciento", "dos", "san",
    "ser", "años", "año", "cada", "muchos", "uno", "aun"
]

stop_words = set(stopwords.words("spanish"))
stop_words.update(palabras_irrelevantes)

# =====================
# Agrupar textos por cluster
# =====================
cluster_texts = df.groupby("cluster_label")["texto_completo"].apply(list).to_dict()

# =====================
# Función para limpiar y tokenizar
# =====================
def limpiar_tokenizar(texto: str):
    texto = texto.lower()
    texto = re.sub(r"\d+", "", texto)  # eliminar números
    palabras = re.findall(r"\b[a-záéíóúñ]{3,}\b", texto)  # tokens >= 3 letras
    palabras = [w for w in palabras if w not in stop_words]
    return palabras

# =====================
# 1) Palabras clave (Top N más frecuentes)
# =====================
n_palabras = 10
palabras_clave = {}

for cluster_id, texts in cluster_texts.items():
    all_words = []
    for txt in texts:
        all_words.extend(limpiar_tokenizar(txt))

    freq = Counter(all_words)
    palabras_clave[cluster_id] = [w for w, _ in freq.most_common(n_palabras)]

df_palabras_clave = pd.DataFrame({k: pd.Series(v) for k, v in palabras_clave.items()})

print("\n=== Palabras clave por cluster (Top 10 más frecuentes) ===")
display(df_palabras_clave)

# =====================
# 2) Palabras distintivas por cluster (exclusivas)
# =====================
top_n_distintivas = 10
palabras_distintivas = {}

# Frecuencias por cluster
cluster_freqs = {}
cluster_sets = {}

for cluster_id, texts in cluster_texts.items():
    all_words = []
    for txt in texts:
        all_words.extend(limpiar_tokenizar(txt))

    cluster_freqs[cluster_id] = Counter(all_words)
    cluster_sets[cluster_id] = set(all_words)

for cluster_id, words_set in cluster_sets.items():
    otras = set().union(*(s for cid, s in cluster_sets.items() if cid != cluster_id))
    exclusivas = words_set - otras

    top_exclusivas = [
        w for w, _ in cluster_freqs[cluster_id].most_common()
        if w in exclusivas
    ][:top_n_distintivas]

    palabras_distintivas[cluster_id] = top_exclusivas

df_palabras_distintivas = pd.DataFrame({k: pd.Series(v) for k, v in palabras_distintivas.items()})

print("\n=== Palabras distintivas por cluster (Top 10 exclusivas) ===")
display(df_palabras_distintivas)
